In [1]:
from langgraph.store.memory import InMemoryStore
import uuid

in_memory_store = InMemoryStore()

In [2]:
user_id = "1"
namespace_for_memory = (user_id, "memories")

In [3]:
memory_id = str(uuid.uuid4())
memory = {"food_preference" : "I like pizza"}
in_memory_store.put(namespace_for_memory, memory_id, memory)

In [4]:
memories = in_memory_store.search(namespace_for_memory)
memories[-1].dict()

{'namespace': ['1', 'memories'],
 'key': '26ce9f74-1c52-4e74-a50c-0f9118644fcb',
 'value': {'food_preference': 'I like pizza'},
 'created_at': '2025-11-21T02:36:50.931377+00:00',
 'updated_at': '2025-11-21T02:36:50.931379+00:00',
 'score': None}

In [5]:
# 语义搜索
# from langchain.embeddings import init_embeddings
from langchain_community.embeddings import DashScopeEmbeddings
store = InMemoryStore(
    index={
        # "embed": init_embeddings
        "embed": DashScopeEmbeddings(model="text-embedding-v3"),
        "dims": 1024,                              # Embedding dimensions
        "fields": ["food_preference", "$"]              # Fields to embed
    }
)

In [7]:
# Store with specific fields to embed
store.put(
    namespace_for_memory,
    str(uuid.uuid4()),
    {
        "food_preference": "I love Italian cuisine",
        "context": "Discussing dinner plans"
    },
    index=["food_preference"]  # Only embed "food_preferences" field
)

# Store without embedding (still retrievable, but not searchable)
store.put(
    namespace_for_memory,
    str(uuid.uuid4()),
    {"system_info": "Last updated: 2024-01-01"},
    index=False
)

In [8]:
memories = store.search(
    namespace_for_memory,
    query="What does the user like to eat?",
    limit=3  # Return top 3 matches
)
memories

[Item(namespace=['1', 'memories'], key='8e8ead8e-35a8-4f38-a36b-6e994a75dc57', value={'food_preference': 'I love Italian cuisine', 'context': 'Discussing dinner plans'}, created_at='2025-11-21T02:39:45.255526+00:00', updated_at='2025-11-21T02:39:45.255530+00:00', score=0.624442150801609),
 Item(namespace=['1', 'memories'], key='27e16c98-8334-44d3-a94b-c637427e711c', value={'system_info': 'Last updated: 2024-01-01'}, created_at='2025-11-21T02:39:45.255878+00:00', updated_at='2025-11-21T02:39:45.255879+00:00', score=None)]